In [1]:
import generator
import time

import numpy as np

from pysat.formula import CNF
from pysat.solvers import Solver


def read_sat(sat_path):
    with open(sat_path) as f:
        sat_lines = f.readlines()
        header = sat_lines[0]
        header_info = header.replace("\n", "").split(" ")
        num_vars = int(header_info[-2])
        num_clauses = int(header_info[-1])

        sat = [[int(x) for x in line.replace(' 0\n', '').split(' ')]
               for line in sat_lines[1:]]

        return sat, num_vars, num_clauses


def sat_transform(sat):
    new_sat = [set([abs(x) for x in clause]) for clause in sat]
    return new_sat


def if_dependent(x, y):
    set_x = set(abs(i) for i in x)
    set_y = set(abs(i) for i in y)
    if set_x.intersection(set_y):
        return True
    else:
        return False


def dependency_analysis(sat, num_vars, num_clauses):
    dependency = [[] for i in range(num_clauses)]

    start_time = time.time()
    inersection_graph = np.zeros([num_clauses, num_vars])
    for idx, clause in enumerate(sat):
        for literal in clause:
            var_idx = abs(literal) - 1
            if literal > 0:
                inersection_graph[idx, var_idx] = 1
            else:
                inersection_graph[idx, var_idx] = -1
    print(f"inersection_graph build time: {time.time() - start_time:.4f}")

    for i in range(num_clauses):
        for j in range(i+1, num_clauses):
            pair_dependency = np.multiply(
                inersection_graph[i], inersection_graph[j])
            if np.any(pair_dependency):
                dependency[i].append(j)
                dependency[j].append(i)

    return dependency


def lll_test(sat, k, log=False):
    num_clauses = len(sat)
    # initial the alpahs for each clause (event)
    event_prob = [2 ** (-len(clause)) for clause in sat]
    # print(f'initial alphas: {alphas}')

    # dependency analysis
    start_time = time.time()
    sat = sat_transform(sat)
    dependency = [[] for i in range(num_clauses)]
    for i in range(num_clauses):
        for j in range(i+1, num_clauses):
            if sat[i].intersection(sat[j]):
                dependency[i].append(j)
                dependency[j].append(i)
    # print(f"dependency analysis time: {time.time() - start_time:.4f}")

    # propagation
    iterations = 0
    alphas = [x for x in event_prob]
    for i in range(k):
        if log:
            print(alphas)
        iterations += 1
        if max(alphas) > 1:
            break
        
        update_alphas = [event_prob[i] / np.prod([1 - alphas[j] for j in dependency[i]]) for i in range(num_clauses)]
        alphas = update_alphas

    return iterations, dependency


In [3]:
from pysat.formula import CNF
from pysat.solvers import Solver


sat_instance = generator.lll_sat_generator(100, 5)

sat_instance = [[1, -2, 3], [1, 2, -3], [1, 2, 3]]
# sat_instance = [[1, -2, 3]]


# sat_path = './industrial_formulas/cmu-bmc-longmult15.processed.cnf'
# sat_instance, num_vars, num_clauses = read_sat(sat_path)


print(f'the size of SAT: {len(sat_instance)}')

print('----------')

iterations, dependency = lll_test(sat_instance, 100, log=True)
print(f'lll_test iterations: {iterations}')

print('----------')

cnf = CNF(from_clauses=sat_instance)
start_time = time.time()
with Solver(bootstrap_with=cnf) as solver:
    print(f'The SAT instance can be solved: {solver.solve()}')
    print(solver.get_model())
print(f"solver time: {time.time() - start_time:.4f}")

the size of SAT: 3
----------
[0.125, 0.125, 0.125]
[0.16326530612244897, 0.16326530612244897, 0.16326530612244897]
[0.17853955978584174, 0.17853955978584174, 0.17853955978584174]
[0.18524082471293415, 0.18524082471293415, 0.18524082471293415]
[0.18830050866379736, 0.18830050866379736, 0.18830050866379736]
[0.1897227737367076, 0.1897227737367076, 0.1897227737367076]
[0.19038939223486365, 0.19038939223486365, 0.19038939223486365]
[0.19070304756521353, 0.19070304756521353, 0.19070304756521353]
[0.19085089593734478, 0.19085089593734478, 0.19085089593734478]
[0.1909206471662798, 0.1909206471662798, 0.1909206471662798]
[0.19095356735823252, 0.19095356735823252, 0.19095356735823252]
[0.1909691075194187, 0.1909691075194187, 0.1909691075194187]
[0.1909764439987602, 0.1909764439987602, 0.1909764439987602]
[0.19097990768304607, 0.19097990768304607, 0.19097990768304607]
[0.19098154298364609, 0.19098154298364609, 0.19098154298364609]
[0.19098231506132804, 0.19098231506132804, 0.19098231506132804]


In [63]:
import os

formulas_path = './formulas/'

for path in os.listdir(formulas_path):
    sat_path = formulas_path + path
    sat_instance, num_vars, num_clauses = read_sat(sat_path)
    iterations, dependency = lll_test(sat_instance, 100)
    if iterations > 2:
        print('<-------->')
        print(path)
        print(f'lll_test iterations: {iterations}')
    

In [64]:
unsolve_path = './unsolve_formulas/'

unsolve_instances = []


for path in os.listdir(formulas_path):
    sat_path = formulas_path + path
    sat_instance, num_vars, num_clauses = read_sat(sat_path)
    unsolve_instances.append(sat_instance)


num_all = len(unsolve_instances)
for i in range(1, 100):
    num_lll = 0
    part_instances = [instance[:i] for instance in unsolve_instances]
    for instance in part_instances:
        iterations, dependency = lll_test(instance, 50)
        if iterations == 50:
            num_lll += 1

    print(f'sat_size: {i}, number of all: {num_all}, number of lll: {num_lll}, percent: {num_lll/num_all}')

sat_size: 1, number of all: 994, number of lll: 994, percent: 1.0
sat_size: 2, number of all: 994, number of lll: 994, percent: 1.0
sat_size: 3, number of all: 994, number of lll: 994, percent: 1.0
sat_size: 4, number of all: 994, number of lll: 994, percent: 1.0
sat_size: 5, number of all: 994, number of lll: 994, percent: 1.0
sat_size: 6, number of all: 994, number of lll: 994, percent: 1.0
sat_size: 7, number of all: 994, number of lll: 994, percent: 1.0
sat_size: 8, number of all: 994, number of lll: 994, percent: 1.0
sat_size: 9, number of all: 994, number of lll: 994, percent: 1.0
sat_size: 10, number of all: 994, number of lll: 994, percent: 1.0
sat_size: 11, number of all: 994, number of lll: 994, percent: 1.0
sat_size: 12, number of all: 994, number of lll: 994, percent: 1.0
sat_size: 13, number of all: 994, number of lll: 994, percent: 1.0
sat_size: 14, number of all: 994, number of lll: 994, percent: 1.0
sat_size: 15, number of all: 994, number of lll: 994, percent: 1.0
sat_